hybrid_bm25_sbert_chatbot (Hybrid BM25 + SBERT: Untuk hasil maksimal.)
Penjelasan:
BM25 mengambil 5 kandidat teratas berdasarkan keyword.
SBERT menghitung similarity semantic antara pertanyaan user dan kandidat.
Top 3 jawaban paling mirip ditampilkan, urut berdasarkan kemiripan semantic.

In [2]:
import json
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, util

In [3]:
# ===== LOAD DATASET =====
with open("data/pertanyaan_jawaban.json", "r", encoding="utf-8") as f:
    faq_data = json.load(f)

faq_questions = [item["pertanyaan"] for item in faq_data]
faq_answers = [item["jawaban"] for item in faq_data]

In [4]:
# ===== INISIALISASI BM25 DAN SBERT =====
tokenized_questions = [q.lower().split() for q in faq_questions]
bm25 = BM25Okapi(tokenized_questions)
sbert_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
faq_embeddings = sbert_model.encode(faq_questions)

In [5]:
def hybrid_search(user_question, bm25_top_n=5, sbert_top_k=3):
    # Step 1. BM25 retrieval
    tokenized_query = user_question.lower().split()
    bm25_scores = bm25.get_scores(tokenized_query)
    bm25_top_indices = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:bm25_top_n]

    # Step 2. SBERT rerank
    candidate_questions = [faq_questions[i] for i in bm25_top_indices]
    candidate_answers = [faq_answers[i] for i in bm25_top_indices]
    candidate_embeddings = [faq_embeddings[i] for i in bm25_top_indices]
    user_embedding = sbert_model.encode([user_question])[0]
    cos_sims = util.cos_sim(user_embedding, candidate_embeddings)[0].cpu().tolist()

    # Pair and rerank
    pairs = list(zip(bm25_top_indices, candidate_questions, candidate_answers, cos_sims))
    pairs = sorted(pairs, key=lambda x: x[3], reverse=True)  # sort by SBERT cosine similarity

    # Ambil top K hasil
    return pairs[:sbert_top_k]

if __name__ == "__main__":
    print("Chatbot Hybrid BM25 + SBERT (FAQ)")
    print("----------------------------------")
    while True:
        user_question = input("\nTulis pertanyaanmu (atau ketik 'selesai' untuk keluar): ")
        if user_question.lower() == "selesai":
            break
        results = hybrid_search(user_question)
        print("\nTop jawaban paling relevan:")
        for i, (idx, q, a, score) in enumerate(results, 1):
            print(f"\n{i}. Pertanyaan mirip: {q}")
            print(f"   Jawaban       : {a}")
            print(f"   Skor kemiripan: {score:.4f}")

Chatbot Hybrid BM25 + SBERT (FAQ)
----------------------------------



Tulis pertanyaanmu (atau ketik 'selesai' untuk keluar):  siapa anda


/Users/abdulkabir/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sentence_transformers/util.py:44: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  a = torch.tensor(a)



Top jawaban paling relevan:

1. Pertanyaan mirip: Apakah Anda merasa leher Anda kaku?
   Jawaban       : Saya mengalami demam tinggi disertai mual.
   Skor kemiripan: 0.2929

2. Pertanyaan mirip: Apakah Anda merasa mata Anda merah atau iritasi?
   Jawaban       : Iya, mata saya terlihat merah dan terasa iritasi.
   Skor kemiripan: 0.2390

3. Pertanyaan mirip: Apakah Anda merasa kulit Anda berubah atau mengalami ruam?
   Jawaban       : Saya mengalami ruam pada kulit dan merasa lelah.
   Skor kemiripan: 0.1901



Tulis pertanyaanmu (atau ketik 'selesai' untuk keluar):  saya mengalami ruam



Top jawaban paling relevan:

1. Pertanyaan mirip: Apakah Anda merasa kulit Anda berubah atau mengalami ruam?
   Jawaban       : Saya mengalami ruam pada kulit dan merasa lelah.
   Skor kemiripan: 0.4650

2. Pertanyaan mirip: Apakah Anda mengalami perdarahan dari gusi atau hidung?
   Jawaban       : Ya, saya sering mengalami pendarahan di gusi dan hidung.
   Skor kemiripan: 0.4076

3. Pertanyaan mirip: Apakah ada ruam atau pembengkakan pada tangan atau kaki?
   Jawaban       : Saya mengalami ruam pada tubuh dan kaki terasa bengkak.
   Skor kemiripan: 0.4013



Tulis pertanyaanmu (atau ketik 'selesai' untuk keluar):  pertanyaan"Bagaimana cara kerja chatbot?"



Top jawaban paling relevan:

1. Pertanyaan mirip: Bagaimana cara kerja chatbot?
   Jawaban       : Chatbot memproses input pengguna dan menghasilkan respons otomatis, bisa berbasis aturan atau AI.
   Skor kemiripan: 0.8943

2. Pertanyaan mirip: Apa itu BERT?
   Jawaban       : BERT adalah model transformer untuk NLP yang dikembangkan oleh Google.
   Skor kemiripan: 0.3369

3. Pertanyaan mirip: Bagaimana cara training model AI?
   Jawaban       : Model AI dilatih dengan data agar dapat mempelajari pola dan membuat prediksi.
   Skor kemiripan: 0.2755



Tulis pertanyaanmu (atau ketik 'selesai' untuk keluar):  selesai
